## New section


This notebook is designed to utilize AWS Bedrock services to perform question-answering tasks using a Retrieval-Augmented Generation (RAG) approach. It employs the 'anthropic.claude-v2' language model.

The notebook can be broadly summarized into the following steps:

1. Configuration and Initialization:
   - Set up the AWS Bedrock client using the provided AWS credentials.
   - Initialize the Bedrock language model (LLM) for generating answers and Bedrock embeddings for handling text embeddings.

2. Loading and Processing Text Data:
   - Load a text file ('askAIevidenceText.txt') which contains the context or evidence to be used for answering questions.
   - Process the loaded text, splitting it into manageable chunks that can be individually embedded and retrieved later.

3. Embedding Generation:
   - Utilize a Sentence Transformer model ('all-MiniLM-L6-v2') to generate embeddings for each chunk of text. These embeddings will later be used to identify the most relevant context for a given question.

4. Question Answering (The RAG Process):
   - For a given question, the notebook does not explicitly show the retrieval part. However, it's implied that the document embeddings would be used to find the most relevant context.
   - Then, using the Claude language model from Anthropic, it generates an answer based on the context provided (or intended to be provided) from the previous step.

The embedding model used, 'amazon.titan-embed-text-v1', is a pre-trained model available through AWS Bedrock services that is capable of converting text into high-dimensional vectors (embeddings).

In [ ]:
!pip install langchain jq sentence-transformers boto3 faiss-cpu


## Create a boto3 client

In [11]:
import json
import os
import sys
import boto3
from langchain.llms import Bedrock

os.environ["AWS_ACCESS_KEY_ID"]="key id here"
os.environ["AWS_SECRET_ACCESS_KEY"]="access key here"

boto3_bedrock = boto3.client(
    service_name="bedrock-runtime",
    region_name="us-west-2",
    aws_access_key_id=os.environ.get("AWS_ACCESS_KEY_ID"),
    aws_secret_access_key=os.environ.get("AWS_SECRET_ACCESS_KEY"),
)
llm = Bedrock(model_id="anthropic.claude-v2", client=boto3_bedrock)


## Creating an embedding using titan

In [3]:
from langchain.embeddings import BedrockEmbeddings

bedrock_embeddings = BedrockEmbeddings(model_id="amazon.titan-embed-text-v1", client=boto3_bedrock)

## Loading documents using the TextLoader and split them into manageable chunks creating embeddings



In [13]:
from langchain.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter, RecursiveCharacterTextSplitter
from pprint import pprint
from sentence_transformers import SentenceTransformer
from langchain.embeddings import BedrockEmbeddings
from langchain.llms.bedrock import Bedrock


encoding = 'ISO-8859-1'
file_path = "askAIevidenceText.txt"
loader = TextLoader(file_path, encoding=encoding)

model = SentenceTransformer('all-MiniLM-L6-v2')
data = loader.load()
text_splitter = RecursiveCharacterTextSplitter(
                chunk_size = 1000,
                chunk_overlap  = 100,
                )
docs = text_splitter.split_documents(data)
# Generate embeddings for all chunks
doc_embeddings = model.encode([doc.page_content for doc in docs])



trying out embedding by titan...

In [ ]:
import numpy as np

sample_embedding = np.array(bedrock_embeddings.embed_query(docs[0].page_content))
print("Sample embedding of a document chunk: ", sample_embedding)
print("Size of the embedding: ", sample_embedding.shape)


In [ ]:
# Using facebook AI support: Facebook AI Similarity Search

In [21]:
import faiss
import numpy as np

def create_index(embeddings):
    dimension = embeddings.shape[1]  # Get the dimension of the embeddings
    index = faiss.IndexFlatL2(dimension)  # Using L2 distance for similarity
    index.add(embeddings)  # Add the embeddings to the index
    return index

def search_index(index, query_embedding, k=5):
    # Faiss expects numpy array in float32
    query_embedding = query_embedding.astype(np.float32)
    # Search for the k nearest neighbors
    distances, indices = index.search(query_embedding, k)
    return indices



In [23]:
# Step 1 & 2: Create a FAISS index and add your document embeddings
faiss_index = create_index(np.array(doc_embeddings).astype(np.float32))

# Step 3 & 4: Create a function to handle new questions
def answer_question(question):
    # Convert the question to an embedding
    question_embedding = model.encode([question])
    question_embedding_np = np.array(question_embedding).astype(np.float32)

    # Retrieve indices of the relevant docs
    indices = search_index(faiss_index, question_embedding_np, k=5)
    relevant_docs = [docs[i] for i in indices[0]]

    # Step 5: Generate the answer with Claude, using the relevant context
    # Join the relevant docs to create a context string
    context = " ".join([doc.page_content for doc in relevant_docs])

    # Generate the response with Claude
    response = llm.generate(
        prompts=[f"Human: {question} return the answer in a list format \n\nRelevant Information: {context}/n Assistant::"]
    )

    return response

new_question = "What are the tools used in action?"
answer = answer_question(new_question)
print(answer)

new_question1 = "What are the city names mentioned?"
answer1 = answer_question(new_question)
print(answer1)



generations=[[Generation(text=' Here are the key tools used in actions mentioned in the evidence:\n\n- Small arms/firearms (used in attack on police station)\n- Sniper rifle (used by sniper in Khost city center attack)  \n- Explosives (referenced for martyr attacks) \n- Chlorine gas (referenced as potential WMD)\n- GSM phones (used for communication)\n- Email (used for communication/coordination)\n\nSo in list format:\n\n- Small arms/firearms\n- Sniper rifle  \n- Explosives\n- Chlorine gas\n- GSM phones\n- Email')]] llm_output=None run=[RunInfo(run_id=UUID('f59cd3f9-4ecf-45f0-b7b5-5e80869b1ca8'))]
generations=[[Generation(text=' - Small arms/firearms\n- Explosives\n- Sniper rifles\n- Mortars\n- Rockets\n- Grenades\n- IEDs (Improvised Explosive Devices)\n- VBIEDs (Vehicle-Borne IEDs) \n- Suicide vests\n- RPGs (Rocket Propelled Grenades)\n- Missiles\n- Artillery\n- Tanks\n- Technical vehicles (weaponized trucks/cars)\n- Drones\n- Heavy machine guns\n- Night vision devices\n- Secure commu

# Load embeddings into FAISS another way with titan




In [ ]:
from langchain.vectorstores import FAISS
import numpy as np
from faiss import IndexFlatL2

docstore = [doc.page_content for doc in docs]
d = 768  # dimension
index_faiss = IndexFlatL2(d)  # You may want to use a more sophisticated index in production

# The index to docstore ID can just be an identity mapping if your ids are 0-indexed
index_to_docstore_id = list(range(len(docs)))

# Now initialize the FAISS vector store with the created index, docstore and mapping
vectorstore_faiss = FAISS(
    embedding_function=lambda x: model.encode([x]),
    index=index_faiss,
    docstore=docstore,
    index_to_docstore_id=index_to_docstore_id
)

# Add the document embeddings to the index
for idx, embedding in enumerate(doc_embeddings):
    # FAISS expects the embeddings to be of type float32
    vectorstore_faiss.add(np.array(embedding, dtype=np.float32).reshape(1, -1), [idx])
